# Run a Model against the Test-Data and Export its Battle Predictions

In [1]:
import pandas as pd
import numpy as np

In [2]:
Submission = catalog.load('Submission')
AvailablePokemons = catalog.load('AvailablePokemons')
BattleResults = catalog.load('Battle_Results')
Weakness_Pokemon = catalog.load('Weakness_Pokemon')
AllPokemons = catalog.load('All_Pokemons')

2020-05-24 10:16:43,863 - kedro.io.data_catalog - INFO - Loading data from `Submission` (CSVDataSet)...
2020-05-24 10:16:43,871 - kedro.io.data_catalog - INFO - Loading data from `AvailablePokemons` (CSVDataSet)...
2020-05-24 10:16:43,875 - kedro.io.data_catalog - INFO - Loading data from `Battle_Results` (CSVDataSet)...
2020-05-24 10:16:46,631 - kedro.io.data_catalog - INFO - Loading data from `Weakness_Pokemon` (CSVDataSet)...
2020-05-24 10:16:46,634 - kedro.io.data_catalog - INFO - Loading data from `All_Pokemons` (CSVDataSet)...


### Create Inference Data

In [3]:
def create_inference_data():
    """Return Same Format as Battle Results with additional PokemonID column"""
    # Repeat every row 6 times
    merged = pd.DataFrame(np.repeat(AvailablePokemons.values,6,axis=0), columns=AvailablePokemons.columns)
    # Drop PokemonID from Submission
    Submission_dropped = Submission.drop(columns=['SelectedPokemonID'])
    # Concat with Submission
    merged = merged.apply(lambda row: pd.concat([row, Submission_dropped.iloc[row.name%6]]), axis=1)
    return merged

In [4]:
inference_data = create_inference_data()

### Run Model against Inference

In [14]:
inference_predictions = np.random.uniform(low=-1.0, high=1.0, size=len(inference_data))

In [15]:
inference_predictions

array([-0.67334653,  0.876185  , -0.00190304, ..., -0.85367629,
        0.22896094,  0.147411  ])

### Run Model against Test Data

In [12]:
from sklearn.metrics import mean_squared_error

In [7]:
BattleResults_Test = catalog.load('Battle_Results_Test')

2020-05-24 10:16:53,199 - kedro.io.data_catalog - INFO - Loading data from `Battle_Results_Test` (CSVDataSet)...


In [8]:
def RelativeResult(X):
    HP = X.apply(lambda row: row["HP_1"] if row["BattleResult"] > 0 else row["HP_2"], axis=1).values
    relativized = X["BattleResult"].values / HP
    return relativized

test_labels = RelativeResult(BattleResults_Test)

In [18]:
test_predictions = np.random.uniform(low=-1.0, high=1.0, size=len(test_labels))
lin_mse = mean_squared_error(test_labels, test_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

1.0537348322623872

### Export

In [39]:
def export(predictions):
    table = np.empty((len(AvailablePokemons), 7))
    for i in range(len(AvailablePokemons)):
        table[i, 0] = i
        table[i, 1:] = predictions[i*6: (i+1)*6]
    df = pd.DataFrame(data=table, columns=['SelectedPokemonID']+list(Submission.Name_2.values)); output
    return df.astype({'SelectedPokemonID': int})
    
output = export(inference_predictions)
output

,SelectedPokemonID,Caterpie,Golem,Krabby,Mewtwo,Raichu,Venusaur
0,0,-0.673347,0.876185,-0.001903,0.044104,-0.088475,-0.505112
1,1,0.331356,-0.544088,-0.307657,-0.095056,0.942036,0.677011
2,2,-0.015797,0.358002,0.620032,-0.089927,0.357616,-0.313250
3,3,-0.178182,0.111369,0.016455,0.698512,-0.363971,-0.096498
4,4,0.646152,-0.913883,-0.551524,0.192226,0.147515,0.325275
...,...,...,...,...,...,...,...
1435,1435,-0.070775,-0.844019,-0.052456,0.827367,-0.387723,-0.636623
1436,1436,-0.721260,0.961378,-0.101890,0.150286,-0.840004,0.786808
1437,1437,-0.712511,0.375991,-0.128202,-0.922079,0.623179,0.227376
1438,1438,-0.306187,0.583784,-0.799536,-0.608873,0.365639,-0.058822


In [42]:
output.to_csv(path_or_buf='../../data/07_model_output/Random/inference_random.csv')